In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ],
    model="gpt-4o-mini"
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

examples = [
    {
    "movie": "Inception",
    "answer": """
🌌🌀🧠
"""
}, 
{
    "movie": "Parasite",
    "answer": """
🏠🌧️🍜
"""
},
{
    "movie": "The Grand Budapest Hotel",
    "answer": """
🏨🎨🕰️
"""
},
{
    "movie": "The Dark Knight",
    "answer": """
🦇🃏🌆
"""
}]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Describe {movie} in three emojis?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

example_prompt = ChatPromptTemplate.from_messages([
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{movie}")
])

find_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

example_chain = RunnablePassthrough.assign(history=load_memory) | example_prompt | chat
find_chain = RunnablePassthrough.assign(history=load_memory) | find_prompt | chat

def invoke_chain(input):
    result = example_chain.invoke({"movie":input})
    memory.save_context({"input": input}, {"output": result.content})

invoke_chain("Top Gun")


✈️🔥🎶

In [50]:
invoke_chain("Godfather")

👨‍👦🍝🔫

In [51]:
invoke_chain("Whiplash")

🥁⚡🎶

In [52]:
invoke_chain("The Avengers")

🦸‍♂️🦸‍♀️🌍

In [53]:
find_chain.invoke({"question":"Please tell me the name of the movie I first asked."})

The movie you first asked about is "Top Gun."

AIMessageChunk(content='The movie you first asked about is "Top Gun."')

In [54]:
invoke_chain("Iron Man")

🤖🦸‍♂️💥

In [55]:
find_chain.invoke({"question":"Please tell me the name of the movie I first asked."})

The movie you first asked about is "The Godfather."

AIMessageChunk(content='The movie you first asked about is "The Godfather."')

In [56]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Godfather'),
  AIMessage(content='👨\u200d👦🍝🔫'),
  HumanMessage(content='Whiplash'),
  AIMessage(content='🥁⚡🎶'),
  HumanMessage(content='The Avengers'),
  AIMessage(content='🦸\u200d♂️🦸\u200d♀️🌍'),
  HumanMessage(content='Iron Man'),
  AIMessage(content='🤖🦸\u200d♂️💥')]}